To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [23]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [24]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [25]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [26]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
unformatted_dataset = load_dataset("Yijia-Xiao/pii-medical_flashcards", split = "train")
dataset = unformatted_dataset.map(formatting_prompts_func, batched = True,)

In [27]:
# Check dataset length (should be the same)
print(f"Original dataset size: {len(unformatted_dataset)}")
print(f"Formatted dataset size: {len(dataset)}")

# Print the first original sample
print("\nOriginal Sample:")
print(f"Instruction: {unformatted_dataset[0]['instruction']}")
print(f"Input: {unformatted_dataset[0]['input']}")
print(f"Output: {unformatted_dataset[0]['output']}")

# Print the corresponding formatted sample
print("\nFormatted Sample:")
print(dataset[0]["text"])

# Verify EOS Token is included
print("\nChecking EOS Token Presence:")
print(dataset[0]["text"].endswith(EOS_TOKEN))  # Should return True

Original dataset size: 33955
Formatted dataset size: 33955

Original Sample:
Instruction: Answer this question truthfully
Input: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Output: Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.

Formatted Sample:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer this question truthfully

### Input:
What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?

### Response:
Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.<|end_of_text|>

Checking EOS Token Presence:
True


In [28]:
# Here is where I will store the pii unigram and bigram tokens
pii_unigrams = set()
pii_bigrams = set()

# Loop through our dataset
for example in unformatted_dataset:
  output_tokens = example["output"].split()
  cleaned_tokens = example["cleaned_output"].split()

  # Iterate over the tokens in the output and cleaned output
  for i, (output_token, cleaned_token) in enumerate(zip(output_tokens, cleaned_tokens)):
    # This if statement is the case where the current token is PII
    # Add the original token from output if it's replaced by a placeholder in cleaned_output
    if cleaned_token.startswith("{{") and cleaned_token.endswith("}}"):
      # Add to unigrams
      pii_unigrams.add(output_token)  # Store original token from output

      # Add to bigrams
      if i < len(output_tokens) - 1:  # Ensure there's a next token in output_tokens
        next_output_token = output_tokens[i + 1]
        pii_bigrams.add((output_token, next_output_token))

    # This if statement is the case where the current token is not PII but it could still be bigram
    # Check if it's a bigram (pair of consecutive tokens)
    if i < len(output_tokens) - 1:  # Ensure there's a next token in output_tokens
      next_output_token = output_tokens[i + 1]
      if i + 1 < len(cleaned_tokens):
        next_cleaned_token = cleaned_tokens[i + 1]

        if next_cleaned_token.startswith("{{") and next_cleaned_token.endswith("}}"):
          pii_bigrams.add((output_token, next_output_token))

print("PII Unigrams:", pii_unigrams)
print("PII Bigrams:", pii_bigrams)

PII Unigrams: {'kidneys.', 'CVD', 'mono-amniotic', 'Malathion', 'Xa', 'contractions.', 'Dantrolene', 'Crocodile', 'Parvovirus', 'V', 'TF', 'arch),', 'CCR5', 'Renshaw', 'MSK', 'E:', 'upstream', 'Nigro', 'TEE', 'ears.', 'placenta', 'PTT', 'Teratogenic', 'Cryptosporidium', 'CD4+', 'Barrett', 'Vinca', 'Media/Tellurite', 'Agoraphobia', 'Apo-E4', 'ABVD', 'Fosamprenavir', 'Triamterene', 'Gardner', 'Craniopharyngiomas', 'Chagas', 'Xanthine', 'Vagus', 'Salk', 'IFN-γ', 'damage.', 'PAD', 'Histoplasmosis', 'TTAGGG', 'Lens', 'Reinke', 'Na+/H2O', 'Atypical', 'Osmotic', 'BUN', 'Stavudine', 'Nafcillin', 'Hepcidin', 'HurLer', '(1/2', 'Bony', '2B', 'intertrigo', 'Agraphia', 'most', 'RSV', 'Neurosyphilis', 'Disseminated', 'Molluscum', 'Maraviroc', 'Ulcers,', 'Hashimoto', 'Ferritin', 'TPN', 'Oprelvekin', 'Anticholinergic/LABA', 'Topiramate', 'Stridor', 'Dronabinol', 'CCKA', 'Trousseau', 'TMP-SMX', 'Laminin', 'NADP+', 'Prenatal', 'Defect', 'Peritonsillar', 'Gynecomastia', 'Staphylococcus', 'Cancer', 'Hepat

In [29]:
#Convert PII unigrams to token IDs
pii_unigrams_IDs = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(token)) for token in pii_unigrams]

# Convert PII bigrams to token IDs (bigrams are tuples of two tokens)
pii_bigrams_IDs = [
    (tokenizer.convert_tokens_to_ids(tokenizer.tokenize(token1)), tokenizer.convert_tokens_to_ids(tokenizer.tokenize(token2)))
    for token1, token2 in pii_bigrams
]

print("PII Unigrams:", pii_unigrams_IDs)
print("PII Bigrams:", pii_bigrams_IDs)

PII Unigrams: [[64381, 36371, 13], [34, 12757], [57501, 33317, 7907, 14546], [30700, 589, 290], [55, 64], [8386, 4109, 13], [35, 519, 1098, 1994], [96141, 83774], [4368, 85, 869, 17402], [53], [11042], [1132, 705], [70246, 20], [49, 729, 71, 675], [4931, 42], [36, 25], [455, 4116], [45, 343, 299], [60079], [7596, 13], [501, 582, 16985], [2898, 51], [21902, 266, 29569], [54467, 437, 4406, 307, 2411], [6620, 19, 10], [3511, 17708], [73092, 936], [12950, 17146, 616, 324, 635], [32, 5746, 1366, 41163], [32, 5481, 13737, 19], [1905, 12757], [37, 437, 1141, 1466, 113765], [22646, 309, 466, 1994], [38, 569, 1215], [16384, 5676, 5237, 661, 983, 72, 23063], [1163, 86152], [55, 32329, 483], [53, 59838], [50, 1727], [2843, 45, 12, 60474], [43965, 13], [44918], [64157, 454, 14833, 8801, 285], [15249, 1929, 23050], [99205], [697, 96973], [16589, 41731, 39, 17, 46], [1688, 89215], [46, 3647, 14546], [33, 1899], [626, 402, 664, 483], [45, 95398, 61597], [39, 752, 21200, 258], [98428, 43, 261], [7, 16

In [30]:
import torch
import torch.nn.functional as F
from trl import SFTTrainer

# The purpose of this code is to modify the usual cross-entropy loss used in training
# language models, so that tokens related to Personally Identifiable Information (PII)
# (both unigrams and bigrams) are penalized more heavily, discouraging the model from
# generating these tokens.

class CustomSFTTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Compute the cross-entropy loss with a penalty for PII tokens (unigrams and bigrams).
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Shift logits and labels for causal language modeling loss
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        # Flatten logits and labels
        flat_logits = shift_logits.view(-1, shift_logits.size(-1))
        flat_labels = shift_labels.view(-1)

        # Initialize a tensor to store the modified logits
        modified_logits = flat_logits.clone()

        # Check which tokens should be penalized (unigrams)
        # is_unigram_penalized is a tensor of True/False values where each position in the tensor
        # corresponds to whether the token at that position should be penalized for being a PII unigram.
        is_unigram_penalized = torch.tensor(
            [label in pii_unigrams_IDs for label in flat_labels.tolist()],
            device=flat_labels.device,
        )

        # For unigrams, apply a penalty to the logits
        penalty_factor = -100  # This will significantly reduce the probability of PII tokens
        modified_logits[is_unigram_penalized] += penalty_factor

        # Check which tokens should be penalized (bigrams)
        # is_bigram_penalized will be a tensor where True indicates that a consecutive pair of
        # tokens should be penalized.
        is_bigram_penalized = torch.zeros_like(is_unigram_penalized, dtype=torch.bool, device=flat_labels.device)
        for i in range(1, len(flat_labels)):
            prev_token = flat_labels[i - 1].item()
            curr_token = flat_labels[i].item()
            if (prev_token, curr_token) in pii_bigrams_IDs:
                is_bigram_penalized[i] = True

        # For bigrams, apply a penalty to the logits
        modified_logits[is_bigram_penalized] += penalty_factor

        # Compute the cross-entropy loss for each token using the modified logits
        token_losses = F.cross_entropy(
            modified_logits, flat_labels, ignore_index=-100, reduction="none"
        )

        # Compute the mean loss, ignoring padding tokens
        loss = token_losses[flat_labels != -100].mean()

        return (loss, outputs) if return_outputs else loss


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [32]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
11.426 GB of memory reserved.


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 33,955 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.013600
2,2.150800
3,1.784300
4,1.480800
5,1.825700
6,1.570300
7,1.394500
8,1.203900
9,0.849800
10,0.847600


In [34]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

324.6855 seconds used for training.
5.41 minutes used for training.
Peak reserved memory = 11.426 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 77.512 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [35]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe next number in the fibonnaci sequence is 13.<|end_of_text|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [36]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
The next number in the fibonnaci sequence is 13.<|end_of_text|>


In [37]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer this question truthfully", # instruction
        "What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer this question truthfully\n\n### Input:\nWhat is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?\n\n### Response:\nVery low Mg2+ levels can cause PTH levels to decrease and Ca2+ levels to increase.<|end_of_text|>']

In [38]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer this question truthfully", # instruction
        "What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer this question truthfully

### Input:
What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?

### Response:
Very low Mg2+ levels cause PTH levels to rise, which can cause Ca2+ levels to be low.<|end_of_text|>


In [41]:
print(unformatted_dataset[:10])

{'input': ['What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?', 'What leads to genitourinary syndrome of menopause (atrophic vaginitis)?', 'What does low REM sleep latency and experiencing hallucinations/sleep paralysis suggest?', 'What are some possible causes of low PTH and high calcium levels?', 'How does the level of anti-müllerian hormone relate to ovarian reserve?', 'What does low Mobility and bulging of TM suggest?', 'What are the possible causes of low glucose and high C-peptide levels?', 'What can cause low Glucose and high C-peptide levels?', 'What is the relationship between low Ejection fraction and dysfunction?', 'What condition is associated with low DLCO?'], 'output': ['Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.', 'Low estradiol production leads to genitourinary syndrome of menopause (atrophic vaginitis).', 'Low REM sleep latency and experiencing hallucinations/sleep paralysis suggests nar

In [42]:
generated_responses = []  # Store generated responses

# Iterate through the first 10 items of the dataset
for i in range(10):
    example = {
        "instruction": unformatted_dataset['instruction'][i],
        "input": unformatted_dataset['input'][i],
        "output": unformatted_dataset['output'][i]
    }

    # Tokenize the instruction-input pair
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                example["instruction"],  # Instruction
                example["input"],  # Input
                "",  # Leave output blank for generation
            )
        ], return_tensors="pt"
    ).to("cuda")

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

    # Decode the response
    output_text = tokenizer.batch_decode(outputs)[0]

    # Extract the actual response
    response = output_text.split("### Response:\n", 1)[-1].strip()
    response = response.replace("<|end_of_text|>", "")  # Clean up

    # Store the response
    generated_responses.append(response)

# Print a few generated responses
for i in range(5):  # Print first 5 responses
    print(f"Instruction: {unformatted_dataset['instruction'][i]}")
    print(f"Input: {unformatted_dataset['input'][i]}")
    print(f"Generated Response: {generated_responses[i]}\n")


Instruction: Answer this question truthfully
Input: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Generated Response: Very low Mg2+ levels can cause PTH levels to be very high and Ca2+ levels to be very low.

Instruction: Answer this question truthfully
Input: What leads to genitourinary syndrome of menopause (atrophic vaginitis)?
Generated Response: Genitourinary syndrome of menopause (atrophic vaginitis) is caused by a decrease in estrogen levels.

Instruction: Answer this question truthfully
Input: What does low REM sleep latency and experiencing hallucinations/sleep paralysis suggest?
Generated Response: Low REM sleep latency and experiencing hallucinations/sleep paralysis suggest that the patient may have a condition known as narcolepsy.

Instruction: Answer this question truthfully
Input: What are some possible causes of low PTH and high calcium levels?
Generated Response: Low PTH and high calcium levels can be caused by a variety of factors,

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
